In [ ]:
from langchain_core.messages import AIMessage, HumanMessage, get_buffer_string
from langchain_core.prompts import format_document
from langchain_core.runnables import RunnableParallel
from operator import itemgetter

from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

from dotenv import load_dotenv, find_dotenv

In [ ]:
load_dotenv(find_dotenv())

In [ ]:
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings

embedding_function = OpenAIEmbeddings()

docs = [
    Document(page_content="the dog loves to eat pizza", metadata={"source": "animal.txt"}),
    Document(page_content="the cat loves to eat lasagna", metadata={"source": "animal.txt"}),
]


db = Chroma.from_documents(docs, embedding_function)
retriever = db.as_retriever()

In [ ]:
from langchain.prompts.prompt import PromptTemplate

_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

In [ ]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
ANSWER_PROMPT = ChatPromptTemplate.from_template(template)

In [ ]:
def format_docs(
    docs: list[Document], document_separator="\n"
):
    doc_strings = [doc.page_content for doc in docs]
    return document_separator.join(doc_strings)

In [ ]:
_inputs = RunnableParallel(
    standalone_question=RunnablePassthrough.assign(
        chat_history=lambda x: get_buffer_string(x["chat_history"])
    )
    | CONDENSE_QUESTION_PROMPT
    | ChatOpenAI(temperature=0)
    | StrOutputParser(),
)
_context = {
    "context": itemgetter("standalone_question") | retriever | format_docs,
    "question": lambda x: x["standalone_question"],
}
conversational_qa_chain = _inputs | _context | ANSWER_PROMPT | ChatOpenAI()

In [ ]:
conversational_qa_chain.invoke(
    {
        "question": "What does the dog love to eat?",
        "chat_history": [],
    }
)

In [ ]:
conversational_qa_chain.invoke(
    {
        "question": "What does the dog love to eat?",
        "chat_history": [
            HumanMessage(content="Who is the cat´s name?"),
            AIMessage(content="Larry"),
        ],
    }
)